In [ ]:
!pip install underthesea


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
import underthesea
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

In [ ]:
#Đưa về kiểu chữ thường
def text_lowercase(text):
    return text.lower()

In [ ]:
#Loại bỏ các con số
def remove_number(text):
    result = re.sub(r'\d+', '', text)
    return result

In [ ]:
#Loại bỏ các dấu câu
def remove_punctuation(text):
    text = text.replace(",", " ").replace(".", " ") \
    .replace(";", " ").replace("“", " ") \
    .replace(":", " ").replace("”", " ") \
    .replace('"', " ").replace("'", " ") \
    .replace("!", " ").replace("?", " ") \
    .replace("-", " ").replace("?", " ")
    return text

In [ ]:
#Loại bỏ khoảng trắng thừa
def remove_whitespace(text):
    return  " ".join(text.split())

In [ ]:
# Bỏ bớt các chữ cái giống nhau liên tiếp( Vd: quaaa -> qua)
def remove_similarletter(text):
    text = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), text, flags=re.IGNORECASE)
    return text

In [ ]:
#Tách từ tiếng Việt sử dụng thư viện underthesea có sẵn
def VN_Tokenize(text,format='text'):
    return underthesea.word_tokenize(text)

In [ ]:
print(VN_Tokenize("hà nội là thủ đô"))


['hà nội', 'là', 'thủ đô']


In [ ]:
#Loại bỏ các stopwords tiếng Việt sử dụng bộ stopwords có sẵn
def remove_VN_stopwords(text):
    file_stopwords = pd.read_csv('/content/vietnamese-stopwords.txt', encoding = 'UTF-8')
    file_stopwords.columns = ["Stop_words"]

    VN_stopword = []
    for i in file_stopwords["Stop_words"]:
        VN_stopword.append(i)

    text_token = VN_Tokenize(text)
    result = [word for word in text_token if word not in VN_stopword]
    return " ".join(result)

In [ ]:
#Chuẩn hóa dữ liệu
def Util(text):
    replace_list = {
       ':v':'hihi', '<3':'yêu', '♥️':'yêu','❤':'yêu','a':'anh','ac':'anh chị','ace':'anh chị em','ad':'quản lý',
       'ae':'anh em','ah':'à','ak':'à','amin':'quản lý','androir':'android','app':'ứng dụng','auto ':'tự động',
       'ây':'vậy','b nào':'bạn nào','bằg':'bằng','băng':'bằng','băp':'bắp','băt':'bắt','battery':'pin','bể':'vỡ',
       'been':'bên','best':'nhất','best':'tốt nhất','bgqafy ':'ngày','bh':'bao giờ','bh':'bây giờ','bhx':'bảo hành',
       'bi':'bị','big':'lớn','bik':'biết','bin':'pin','bit':'biết','bít':'biết','bn':'bạn','bông tróc':'bong tróc', 'k': 'không', 'ok': 'được',
       'bro':'anh em','bt':'bình thường','bt':'biết','bth':'bình thường','bthg':'bình thường','bua':'bữa','bùn':'buồn',
       'buonc':'buồn','bx':'hộp','bye':'tạm biệt','c':'chị','cac':'các','cam':'máy ảnh','card':'thẻ','châu':'khỏe',
       'chiệu':'triệu','chíp':'chip','chội':'trội','chs':'chơi','chửa':'chữa','chug ':'chung','chup':'chụp','chuq':'chung',
       'clip':'đoạn phim','cmt':'bình luận','co':'có','cở':'cỡ','cọc':'cột','cpu':'chíp xử lý','cty':'công ty',
       'cua':'của','cũg':'cũng','cug ':'cũng','cuh':'cũng','cùi':'tệ','củng':'cũng','cụt':'cục','cv':'công việc',
       'cx':'cũng','đ':' đồng','dag':'đang','dăng':'văng','dấp':'lỗi','dất':'rất','đay':'đấy','đâỳ':'đầy','đc':'được',
       'dè':'rè','dể':'dễ','delay':'trễ','dêm':'đêm','đén':'đến','deplay ':'chậm','deu':'đều','diem':'điểm','dien':'diện',
       'đien':'điển','điễn':'điển','dienmayxanh':'điện máy xanh','dín':'dính','dis':'văng','diss':'văng','dk':'được',
       'dmx':'điện máy xanh','dô':'vào','dõ':'rõ','dỡ':'dở','đỗi':'đổi','download':'tải','drop':'tụt','dt':'điện thoại',
       'đt':'điện thoại','đth':'điện thoại','đthoai':'điện thoại','du':'dù','dùg':'dùng','dừg':'dừng','đứg':'đứng',
       'dụg ':'dụng','dung':'dùng','đụng':'chạm','đươc':'được','đuọc ':'được','đưowjc':'được','dựt ':'giật','dx':'được'
       ,'đx':'được','đy':'đi','e':'em','ế':'không bán được','êm':'tốt','f':'facebook','fabook':'facebook',
       'face':'facebook','fast':'nhanh','fb':'facebook','fim':'phim','fix':'sửa','flash sale':'giảm giá','fm':'đài',
       'for what':'vì sao','fps':'tốc độ khung hình','full':'đầy','future':'tương lai','game':'trò chơi','gem':'trò chơi',
       'geme':'trò chơi','gia tiên':'giá tiền','giât':'giật','giốg ':'giống','giử':'dữ','giùm':'dùm','gmae':'trò chơi',
       'gởi':'gửi','gold':'vàng','gơn':'hơn','good':'tốt','good jup':'tốt','gop':'góp','gửa':'gửi','gủng':'cái','h':'giờ',
       'haiz':'thở dài','hẵn ':'hẳn','hành':'hành','hazzz':'haizz','hc':'học','hcm':'hồ chí minh','hd':'chất lượng cao',
       'hdh':'hệ điều hành','hđh':'hệ điều hành','headphone':'tai nghe','hên':'may mắn','hẻo':'yếu','hẹo':'yếu','het':'hết',
       'hét':'hết','hic':'khóc','hieu':'hiểu','high-tech':'công nghệ cao','hít':'sử dụng','hiu':'hiểu','hỉu':'hiểu',
       'hk':'không','hn':'hà nội','hnay':'hôm nay','hoài':'nhiều lần','hoi':'hơi','hới':'hơi','hời':'tốt',
       'hoi han':'hối hận','hok':'không','hong':'không','hông':'không','hot':'nổi bật','hqua':'hôm qua','hs':'học sinh',
       'hssv':'học sinh sinh viên','hut':'hút','huway ':'huawei','huwei ':'huawei','í':'ý','I like it':'tôi thích nó',
       'ik':'đi','ip':'iphone','j':'gì','k':'không','kàm':'làm','kb':'không biết','kg':'không','kh ':'khách hàng',
       'khach':'khách hàng','khát phục':'khắc phục','khj':'khi','khoá ':'khóa','khóai ':'thích','khoẻ':'khỏe',
       'khoẽ':'khỏe','khôg':'không','khoi đong':'khởi động','khong':'không','khoong ':'không','khuân':'khuôn',
       'khủg':'khủng','kím':'kiếm','kipo':'tiêu cực','ko':'không','kt':'kiểm tra','ktra':'kiểm tra','la':'là',
       'lác':'lỗi','lắc':'lỗi','lag':'lỗi','laii':'lại','lak':'giật','lan':'lần','lãng':'giật','lap':'máy tính',
       'laptop':'máy tính','lay':'này','len toi':'lên tới','les':'led','lg':'lượng','lí':'lý','lien':'liên',
       'like':'thích','liti':'nhỏ','live stream':'phát sóng trực tiếp','lm':'làm','ln':'luôn','loadd':'tải ',
       'lôi':'lỗi','lổi':'lỗi','LOL ':'trò chơi','lởm':'kém chất lượng','lỏng lẽo':'lỏng lẻo','luc':'lúc','lun':'luôn',
       'luong':'lượng','luot':'lướt','lưot ':'lượt','m':'mình','mạ':'trời','mắc công':'mất công','macseger':'messenger',
       'mag':'màn','main':'chính','mak':'mà','man':'màn','màng':'màn','màng hình':'màn hình','mao ':'mau','mẩu':'mẫu',
       'mầu ':'màu','max':'lớn nhất','may':'máy','mèn':'màn','méo gì':'làm gì','mih':'mình','mìk':'mình','min':'nhỏ nhât',
       'mìn':'mình','mjh':'mình','mjk':'mình','mjnh':'minh','mk':'mình','mn':'mọi người','mng ':'mọi người','mo':'đâu',
       'mò':'tìm','mobile':'điện thoại','mog':'mong','moi':'mới','mơi':'mới','ms':'mới','mún':'muốn','mước':'mức',
       'mược':'mượt','muot':'mượt','mỷ':'mỹ','n':'nó','n':'nói chuyện','nãn':'nản','nayd':'này','nc':'nói chuyện',
       'nch':'nói chuyện','nch':'nói chung','nếo ':'nếu','ng':'người','ngan':'ngang','nge':'nghe','nghiêm':'nghiệm',
       'ngĩ':'nghĩ','ngốn':'sử dụng','nguon':'nguồn','nhah':'nhanh','nhan vien':'nhân viên','nhay':'nhạy','nhe':'nhé',
       'nhèo':'nhòe','nhiet':'nhiệt','nhiểu':'nhiều','nhiu':'nhiều','nhìu':'nhiều','nhoè':'nhòe','như v':'như vậy',
       'nhug':'nhưng','nhưg':'nhưng','nhữg':'những','nhung':'nhưng','nhuoc':'nhược','nhượt':'nhược','nock ao':'hạ gục',
       'noi':'nói','nống':'nóng','not':'lưu ý','ns ':'nói','nsx':'ngày sản xuất','nt':'nhắn tin','ntin':'nhắn tin',
       'ntn':'như thế nào','nũa':'nữa','nut ':'nút','nv':'nhân viên','nz':'như vậy','ô xi':'oxy','ofice':'văn phòng',
       'ok':'được','ôk':'được','oke':'được','okee':'được','oki':'được','okie':'được','onl':'sử dụng',
       'ộp ẹp':'không chắc chắn','option':'tùy chọn','or':'hoặc','out':'thoát','oỳ':'rồi','pải':'phải','phảm':'phẩm',
       'phẩn':'phẩm','phan van':'phân vân','phèo':'vậy','phut ':'phút','pít':'biết','pro':'chất lượng cao','pùn':'buồn',
       'pv':'giới thiệu','qá':'quá','qc':'quảng cáo','qtv':'quản trị viên','qua ve':'qua vẻ','quang trọng':'quan trọng',
       'qus':'quá','r ':'rồi','rat':'rất','rát':'rất','rắt':'rất','rata':'rất','rễ':'dễ','rep':'trả lời',
       'research':'nghiên cứu','reset':'cài đặt lại','restart':'khởi động lại','review':'đánh giá','rì':'gì',
       'rinh':'mua','rỏ':'rõ','rùi':'rồi','rùng':'dùng','s':'sao','sac':'sạc','sài':'xài','sài':'dùng','sale':'giảm giá',
       'sale off':'giảm giá','sâng':'sáng','sạt':'sạc','saving':'tiết kiệm','sd':'sử dụng','sdt':'số điện thoại',
       'seal':'mới','search':'tìm kiếm','sefil':'chụp ảnh','selfie':'chụp ảnh','setting':'cài đặt','setup':'cài đặt',
       'sexy':'quyến rũ','shiper':'nhân viên giao hàng','shop':'cửa hàng','skill':'kỹ năng','smooth':'mượt',
       'so good':'rất tốt','sp':'sản phẩm','sphẩm':'sản phẩm','stars':'sao','sử':'xử','suất':'xuất','sưj':'sự',
       'sước':'xước','super':'siêu','support':'hỗ trợ','sụt':'tụt','sv':'sinh viên','sx':'sản xuất','t':'tôi',
       'T G D Đ':'thế giới di động','tằm ':'tầm','tes':'kiểm tra','test':'kiểm tra','tet':'tết','teung':'trung',
       'tg':'thời gian','tgdd':'thế giới di động','tgdđ':'thế giới di động','thag':'tháng','thág':'tháng','ship':'giao','Ship':'giao',
    }
    text = text.split()
    len_ = len(text)
    for i in range(0, len_):
        for k, v in replace_list.items():
            if (text[i]==k):
                text[i] = v
    return " ".join(text)

In [ ]:
emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"
                u"\U0001F300-\U0001F5FF"
                u"\U0001F680-\U0001F6FF"
                u"\U0001F1E0-\U0001F1FF"
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE) # Unicode emojis.

In [ ]:
def Text_PreProcessing_util(data):
    result_one = []
    for i in data:
        i = str(i)
        text = text_lowercase(i)
        text = re.sub(emoji_pattern, " ", text)
        text = remove_similarletter(text)
        text = remove_number(text)
        text = remove_punctuation(text)
        text = remove_whitespace(text)
        text = remove_VN_stopwords(text)
        text = Util(text)
        result_one.append(text)
    return result_one

In [ ]:
!pip3 install vncorenlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 59.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645932 sha256=1f17a2e43357a69dede3f9e02a3244b7ae79b01f9342fed6e4468f5fec44906a
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp


In [ ]:
def word_segmentation(sentence):
    word_sentence = annotator.tokenize(sentence)
    if len(word_sentence) > 0:
        return " ".join(word_sentence[0])
    else:
        return ""

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset
from torchvision.transforms import v2
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
import tensorflow as tf

In [ ]:
data = pd.read_csv("/content/data_huy_ver1.csv")
data.dropna(inplace=True)
data

NameError: name 'pd' is not defined

In [ ]:
X_data = data["comment"]
Y_data_one = data["label"]
Y_data = []
for label in Y_data_one:
    if label == "Negative":
        Y_data.append(-1)
    elif label == "Neutral":
        Y_data.append(0)
    elif label == "Positive":
        Y_data.append(1)
    else:
        print(label)
X_data = Text_PreProcessing_util(X_data)

In [ ]:
def one_hot_labels(Y):
    label = np.zeros(3,)
    label[Y+1] = 1
    return label

In [ ]:
Y_data = [one_hot_labels(y) for y in Y_data]


In [ ]:
BATCH_SIZE = 128
MAX_SEQ_LEN = 128
MAX_FEATURES = 5000 # vocab size
EMBEDDING_DIMS = 64
RANDOM_SEED = 123
LR=1e-4
EPOCHS=30
checkpoint_root = '/kaggle/working/'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
phobert = "vinai/phobert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(phobert)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Cài đặt Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Tải VnCoreNLP
!wget -qO - https://github.com/vncorenlp/VnCoreNLP/archive/refs/tags/v1.1.1.tar.gz | tar zx
!mv VnCoreNLP-1.1.1 VnCoreNLP

# Tải mô hình VnCoreNLP
!wget -P VnCoreNLP https://github.com/vncorenlp/VnCoreNLP/blob/master/VnCoreNLP-1.1.1.jar


--2024-11-05 03:18:52--  https://github.com/vncorenlp/VnCoreNLP/blob/master/VnCoreNLP-1.1.1.jar
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘VnCoreNLP/VnCoreNLP-1.1.1.jar.1’

VnCoreNLP-1.1.1.jar     [ <=>                ] 283.12K  --.-KB/s    in 0.04s   

2024-11-05 03:18:52 (7.58 MB/s) - ‘VnCoreNLP/VnCoreNLP-1.1.1.jar.1’ saved [289911]



In [ ]:
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP("/content/VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g')

def word_segmentation(sentence):
    word_sentence = annotator.tokenize(sentence)
    if len(word_sentence) > 0:
        return " ".join(word_sentence[0])
    else:
        return ""
class CustomDataset(Dataset):
    def __init__(self, X, Y, tokenizer, max_len):
        self.X = []
        self.Y = []
        for i in range(len(X)):
            segment = word_segmentation(X[i])
            if len(segment) > 0:
                self.X.append(segment)
                self.Y.append(Y[i])
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        sentence = self.X[idx]
        label = self.Y[idx]
        encoding = self.tokenizer.encode_plus(
            sentence,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors='pt',
        )
        return {
            'sentence': sentence,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_masks': encoding['attention_mask'].flatten(),
            'label': label,
        }

In [ ]:
train_dataset = CustomDataset(X_train, y_train, tokenizer, MAX_SEQ_LEN)
val_dataset = CustomDataset(X_test, y_test, tokenizer, MAX_SEQ_LEN)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [ ]:
class BertClassifier(nn.Module):
    def __init__(self):
        super(BertClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("vinai/phobert-base-v2")
        self.drop = nn.Dropout(p=0.3)
        self.fc = nn.Sequential(
            nn.Linear(in_features=self.bert.config.hidden_size, out_features=32),
            nn.ReLU(),
            nn.LayerNorm(32),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=32, out_features=8),
            nn.ReLU(),
            nn.LayerNorm(8),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=8, out_features=3),
            nn.Softmax(1)
        )

    def forward(self, input_ids, attention_mask):
        last_hidden_state, output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
#             token_type_ids=token_type_ids,
            return_dict=False
        )

        x = self.drop(output)
        x = self.fc(x)
        return x

In [ ]:
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
def train(train_dataloader, model, loss_fn, optimizer, epochs=50, val_dataloader=None, validation=False, checkpoint=False, checkpoint_path="", save_folder="model", save_metrics="val_accuracy"):
    min_val_loss = 9999
    max_val_acc = 0
    last_epoch = 0
    model = model.to(device)
    loss_fn = loss_fn.to(device)
    if checkpoint:
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        last_epoch = checkpoint['epoch']
        val_loss = checkpoint['loss']
        min_val_loss = val_loss.item()
    num_epochs = last_epoch+epochs
    history = {
        "loss": [],
        "accuracy": [],
        "val_loss": [],
        "val_accuracy": [],
    }
    for epoch in range(last_epoch, num_epochs):
        model.train()
        process_bar = tqdm(train_dataloader)
        total_loss = 0
        total_acc = 0
        count = 0
        for step, data in enumerate(process_bar):
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_masks'].to(device)

            labels = data['label'].to(device)
            output = model(
              input_ids=input_ids,
              attention_mask=attention_mask,
            )
            train_loss = loss_fn(output, labels)
            train_loss.backward()
#             nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            optimizer.zero_grad()

            pred = torch.argmax(output, 1)-1
            labels = torch.argmax(labels, 1)-1
            train_acc = (labels == pred).sum()/len(labels)
            del input_ids, attention_mask, labels
            process_bar.set_description("Epoch: {}/{}, Loss: {:0.8f}, Accuracy: {:0.2f}".format(epoch+1, num_epochs, train_loss.item(), train_acc))

            total_loss += train_loss.item()
            total_acc += train_acc.cpu()
            count += 1
        history["loss"].append(total_loss/count)
        history["accuracy"].append(total_acc/count)
        if validation:
            all_losses = []
            all_predictions = []
            all_labels = []
            model.eval()

            with torch.no_grad():
                for iter, data in enumerate(val_dataloader):
                    input_ids = data['input_ids'].to(device)
                    attention_mask = data['attention_masks'].to(device)
                    labels = data['label'].to(device)
                    optimizer.zero_grad()
                    output = model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                    )
                    val_loss = loss_fn(output, labels)
                    pred = torch.argmax(output, 1)-1
                    labels = torch.argmax(labels, 1)-1
                    all_labels.extend(labels.tolist())
                    all_predictions.extend(pred)
                    all_losses.append(val_loss.item())
                    del input_ids, attention_mask, labels
                acc = (torch.Tensor(all_labels) == torch.Tensor(all_predictions)).sum()/len(all_labels)
                total_val_loss = sum(all_losses)/len(all_losses)
                if save_metrics == 'val_accuracy' and acc > max_val_acc:
                    max_val_acc = acc
                    torch.save(model, "{}/{}.pt".format(save_folder, epoch+1))
                    torch.save(model, "best.pt".format(save_folder, epoch+1))
                elif total_val_loss < min_val_loss:
                    min_val_loss = total_val_loss
                    torch.save(model, "{}/{}.pt".format(save_folder, epoch+1))
                    torch.save(model, "best.pt".format(save_folder, epoch+1))
                print("Epoch: {}/{}, loss: {:0.8f}, accuracy: {:0.8f}, val_loss: {:0.8f}, val_accuracy: {:0.8f}".format(epoch+1, num_epochs, total_loss/count, total_acc/count, total_val_loss, acc))
                history["val_loss"].append(total_val_loss)
                history["val_accuracy"].append(acc.cpu())
    return history

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
def eval(model, val_dataloader):
    all_predictions = []
    all_labels = []
    model = model.to(device)
    model.eval()

    with torch.no_grad():
        for iter, data in enumerate(val_dataloader):
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_masks'].to(device)

            labels = data['label'].to(device)
            output = model(
              input_ids=input_ids,
              attention_mask=attention_mask,
            )
            pred = torch.argmax(output, 1)-1
            labels = torch.argmax(labels, 1)-1

            all_labels.extend(labels.tolist())
            all_predictions.extend(pred)
            del input_ids, attention_mask, labels
        all_predictions = [x.item() for x in all_predictions]
        print(classification_report(all_labels, all_predictions))
        return all_labels, all_predictions

In [ ]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [ ]:
loss_fn = nn.CrossEntropyLoss()
classifier = BertClassifier()
optimizer = torch.optim.AdamW([
    {"params": classifier.bert.parameters()},
    {"params": classifier.fc.parameters()}
], lr=3e-5, weight_decay = 2e-5)

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
history = train(train_dataloader, model = classifier, loss_fn=loss_fn, optimizer=optimizer, epochs=EPOCHS, val_dataloader=val_dataloader, validation=True, save_folder="/content/sample_data")

  0%|          | 0/155 [00:00<?, ?it/s]

Epoch: 1/30, loss: 0.99445219, accuracy: 0.56400198, val_loss: 0.92489797, val_accuracy: 0.67192876


  0%|          | 0/155 [00:00<?, ?it/s]

Epoch: 2/30, loss: 0.94157027, accuracy: 0.65585685, val_loss: 0.91743792, val_accuracy: 0.67941713


  0%|          | 0/155 [00:00<?, ?it/s]

...

  0%|          | 0/155 [00:00<?, ?it/s]

Epoch: 5/30, loss: 0.90132109, accuracy: 0.71363908, val_loss: 0.90025621, val_accuracy: 0.69338191


  0%|          | 0/155 [00:00<?, ?it/s]

Epoch: 6/30, loss: 0.89168444, accuracy: 0.72739917, val_loss: 0.88922699, val_accuracy: 0.70410848


  0%|          | 0/155 [00:00<?, ?it/s]

In [ ]:
model = torch.load("/content/best.pt")
labels, preds = eval(model, val_dataloader)

<ipython-input-40-7ba0c04a39a9>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("/content/best.pt")


              precision    recall  f1-score   support

          -1       0.81      0.86      0.83       780
           0       0.88      0.81      0.84       790
           1       0.84      0.86      0.85       833

    accuracy                           0.84      2403
   macro avg       0.84      0.84      0.84      2403
weighted avg       0.84      0.84      0.84      2403

